In [1]:
pip install faster-whisper

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 MB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 3.0 MB/s eta 0:00:00


In [3]:
pip install noisereduce

In [ ]:
import librosa, soundfile as sf
import noisereduce as nr
import numpy as np
from pydub import AudioSegment, silence

# 1️⃣ Load audio
audio = AudioSegment.from_file("my_lazada_sample1.wav")
audio = audio.set_channels(1).set_frame_rate(16000)

# 2️⃣ Remove long silences (>800ms below -35dB)
nonsilent = silence.split_on_silence(
    audio,
    min_silence_len=800,
    silence_thresh=audio.dBFS - 35,
    keep_silence=200  # keep some padding
)
cleaned_audio = AudioSegment.silent(duration=0)
for chunk in nonsilent:
    cleaned_audio += chunk
cleaned_audio.export("temp_nosilence.wav", format="wav")

# 3️⃣ Noise reduction (spectral gating)
y, sr = librosa.load("temp_nosilence.wav", sr=16000)
reduced_noise = nr.reduce_noise(y=y, sr=sr,
                                 prop_decrease=1.0,       # Amount of noise reduction (0.0–1.0)
    stationary=True,         # True for constant background noise
    use_tqdm=False,          # Show progress bar
    n_fft=1024,              # FFT window size
    win_length=1024,         # Window length
    hop_length=256           # Hop size
          )

sf.write("sample_denoised.wav", reduced_noise, sr)
print("✅ Saved denoised, silence-removed audio as 'sample_denoised.wav'")

/usr/local/lib/python3.12/dist-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(flt)p?( \(default\))?$', token):
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:314: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(dbl)p?( \(default\))?$', token):


In [3]:
from faster_whisper import WhisperModel

model = WhisperModel("medium", device="cpu", compute_type="int8")

segments, info = model.transcribe("sample_denoised.wav")
for segment in segments:
    print("[%.2fs -> %.2fs] %s" % (segment.start, segment.end, segment.text))

KeyboardInterrupt: 